In [4]:
import time
import pandas as pd
import numpy as np

d1 = pd.read_csv("data/材料1.csv")
d2 = pd.read_csv("data/材料2.csv")
d3 = pd.read_csv("data/材料3.csv")
d4 = pd.read_csv("data/材料4.csv")

d1['材料'] = 1
d2['材料'] = 2
d3['材料'] = 3
d4['材料'] = 4

pd.set_option('future.no_silent_downcasting', True)

data = pd.concat([d1, d2, d3, d4])
data['励磁波形'] = data['励磁波形'].replace({'正弦波': 1, '三角波': 2, '梯形波': 3})

B_ = data.iloc[:, 4:-1].values

lst = ['温度', '频率', "磁芯损耗", '励磁波形']
for i in range(1024):
    lst.append(i)
lst.append('材料')
data.columns = lst

# std_flux = np.std(magnetic_flux_density, axis=1)  # 标准差
max_flux = np.max(B_, axis=1)  # 峰值
min_flux = np.min(B_, axis=1)
# delta_flux = max_flux - min_flux
# sqrt_root = np.mean(B_, axis=1)  # 均方根
# ave_flux = np.mean(B_, axis=1)
# sqrt_ave = sqrt_root / ave_flux

data['B_max'] = max_flux
data['B_min'] = min_flux
# data['delta_B'] = delta_flux
# data['B_rms'] = sqrt_root
# data['B_ave'] = ave_flux
# data['k_f'] = sqrt_ave

filter_data = data[
    ['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min', '磁芯损耗']
]

# to = pd.DataFrame(filter_data)
# to.to_csv("~/Desktop/filter_data.csv", index=False)
filter_data.head()

,温度,频率,励磁波形,材料,B_max,B_min,磁芯损耗
0,25,50030,1,1,0.028849,-0.028840,1997.955250
1,25,50020,1,1,0.031419,-0.031427,2427.749830
2,25,50020,1,1,0.035535,-0.035513,3332.725760
3,25,50020,1,1,0.040015,-0.040025,4502.908007
4,25,50030,1,1,0.045028,-0.045085,6063.023248


In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

start_time = time.time()
X = data[['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min']]  # 自变量 
y = data['磁芯损耗']  # 因变量

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = ExtraTreesRegressor(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
end_time = time.time()

mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
mse = mean_squared_error(y_test, y_pred)

print(f"mape: {mape:.2f}, mse: {mse:.2f}")

mape: 9.16, mse: 1365179387.41


In [8]:
from deap import base, creator, tools, algorithms
import random
import numpy as np
import pandas as pd

# 创建适应度函数（多目标：损耗最小化，传输能量最大化）
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))  # 目标函数都是最小化
creator.create("Individual", list, fitness=creator.FitnessMulti)

# 初始化个体和种群
def init_individual():
    return creator.Individual([random.uniform(25, 90),  # 温度
                               random.uniform(50000, 500000),  # 频率
                               random.randint(1, 3),  # 波形 (ensure integers)
                               random.randint(1, 4),  # 材料 (ensure integers)
                               random.uniform(0.01, 0.1),  # B_max
                               random.uniform(-0.1, -0.01)])  # B_min

toolbox = base.Toolbox()
toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 定义适应度函数
def evaluate(individual):
    T, f, waveform, material, B_max, B_min = individual

    # Ensure B_max > B_min
    if B_max <= B_min:
        B_max, B_min = B_min, B_max

    x = pd.DataFrame([[T, f, waveform, material, B_max, B_min]],
                     columns=['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min'])

    # 计算损耗
    y_pred = model.predict(x)[0]

    # Ensure y_pred is a real number
    y_pred = np.real_if_close(y_pred, tol=1000)

    # 传输能量的目标函数
    transmission_energy = f * B_max

    # Ensure transmission_energy is a real number
    transmission_energy = np.real_if_close(transmission_energy, tol=1000)

    # Debugging statements
    # print(f"Evaluated Individual: {individual}")
    # print(f"y_pred: {y_pred}, type: {type(y_pred)}")
    # print(f"transmission_energy: {transmission_energy}, type: {type(transmission_energy)}")

    return y_pred, -transmission_energy  # 损耗最小化，传输能量最大化

toolbox.register("evaluate", evaluate)

# Use appropriate mutation operators for integers
def mut_individual(individual):
    # Mutate continuous variables
    continuous_vars = [0, 1, 4, 5]
    for idx in continuous_vars:
        individual[idx] += random.gauss(0, 1)
        # Ensure variables stay within bounds
        if idx == 0:
            individual[idx] = min(max(individual[idx], 25), 90)
        elif idx == 1:
            individual[idx] = min(max(individual[idx], 50000), 500000)
        elif idx == 4:
            individual[idx] = min(max(individual[idx], 0.01), 0.1)
        elif idx == 5:
            individual[idx] = min(max(individual[idx], -0.1), -0.01)

    # Mutate integer variables
    integer_vars = [2, 3]
    for idx in integer_vars:
        if random.random() < 0.2:
            if idx == 2:
                individual[idx] = random.randint(1, 3)
            elif idx == 3:
                individual[idx] = random.randint(1, 4)
    return individual,

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", mut_individual)
toolbox.register("select", tools.selNSGA2)
toolbox.register("map", map)

# 运行优化
def optimize():
    population = toolbox.population(n=100)
    algorithms.eaMuPlusLambda(population, toolbox, mu=100, lambda_=200, cxpb=0.7, mutpb=0.2, ngen=40, verbose=True)

    return population

best_solutions = optimize()

/opt/anaconda3/envs/py39/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/anaconda3/envs/py39/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	100   
1  	181   
2  	176   
3  	182   
4  	179   
5  	178   
6  	170   
7  	178   
8  	184   
9  	184   
10 	182   
11 	181   
12 	184   
13 	184   
14 	187   
15 	177   
16 	175   
17 	180   
18 	186   
19 	176   
20 	173   
21 	181   
22 	185   
23 	175   
24 	178   
25 	176   
26 	178   
27 	182   
28 	177   
29 	181   
30 	179   
31 	177   
32 	179   
33 	173   
34 	188   
35 	179   
36 	184   
37 	188   
38 	181   
39 	184   
40 	182   
